In [1]:
import numpy as np
import pandas as pd

In [2]:
import kagglehub

path = kagglehub.dataset_download("oddrationale/mnist-in-csv")

C:\Users\vgrig\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_csv = pd.read_csv(path + "/mnist_train.csv")
test_csv = pd.read_csv(path + "/mnist_test.csv")
train_2_csv = pd.read_csv("./data/train.csv")
train_csv

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
x_train = np.array(train_csv.iloc[:, 1:].values)
x_2_train = np.array(train_2_csv.iloc[:, 1:].values)
y_train = np.array(train_csv.iloc[:, 0].values)
y_2_train = np.array(train_2_csv.iloc[:, 0].values)
x_train = np.array(np.concatenate([x_train.data, x_2_train.data], axis=0))
y_train = np.array(np.concatenate([y_train.data, y_2_train.data], axis=0))
print(x_train.shape)
x_train = x_train / 255.0
x_test = np.array(test_csv.iloc[:, 1:].values)
y_test = np.array(test_csv.iloc[:, 0].values)
x_test = x_test / 255.0
print(x_test.shape)

(102000, 784)
(10000, 784)


In [5]:
def to_one_hot(y, num_classes):
    return np.eye(num_classes)[y]


y_train = to_one_hot(y_train, 10)
y_test = to_one_hot(y_test, 10)

In [6]:
np.random.seed(42)
class Linear:
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        self.weights = np.random.randn(in_features, out_features) * np.sqrt(2 / in_features)
        self.bias = np.zeros((1, out_features))
        self.input = None

    def forward(self, x):
        self.input = x
        return np.dot(x, self.weights) + self.bias

    def backward(self, grad_output, learning_rate):
        grad_input = np.dot(grad_output, self.weights.T)
        grad_weights = np.dot(self.input.T, grad_output)
        grad_bias = np.sum(grad_output, axis=0, keepdims=True)
        self.weights -= learning_rate * grad_weights
        self.bias -= learning_rate * grad_bias
        return grad_input

In [7]:
class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x > 0).astype(float)
        return np.maximum(x, 0)

    def backward(self, grad_output):
        return grad_output * self.mask

In [8]:
class Tanh:
    def __init__(self):
        self.output = None

    def forward(self, x):
        self.output = np.tanh(x)
        return self.output

    def backward(self, grad_output):
        return (1 - self.output ** 2) * grad_output

In [9]:
class Dropout:
    def __init__(self, dropout_rate=0.5):
        self.dropout_rate = dropout_rate
        self.mask = None

    def forward(self, x, is_training=True):
        if is_training:
            self.mask = np.random.binomial(1, 1 - self.dropout_rate, size=x.shape)
            return x * self.mask / (1 - self.dropout_rate)
        else:
            return x

    def backward(self, grad_output):
        return grad_output * self.mask / (1 - self.dropout_rate)

In [10]:
class Softmax:
    def __init__(self):
        self.output = None

    def forward(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        self.output = exp_x / np.sum(exp_x, axis=1, keepdims=True)
        return self.output

    def backward(self, grad_output):
        grad_input = self.output * (grad_output - np.sum(self.output * grad_output, axis=1, keepdims=True))
        return grad_input

In [11]:
import inspect


class Sequential:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, x, is_training=True):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, is_training)
            else:
                x = layer.forward(x)
        return x

    def backward(self, grad_output, learning_rate):
        for layer in reversed(self.layers):
            if hasattr(layer, 'backward'):
                if 'learning_rate' in inspect.signature(layer.backward).parameters:
                    grad_output = layer.backward(grad_output, learning_rate)
                else:
                    grad_output = layer.backward(grad_output)

In [12]:
def cross_entropy_loss(y_pred, y_true):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    loss = -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
    grad = (y_pred - y_true) / y_true.shape[0]
    return loss, grad

In [13]:
y_test_labels = np.argmax(y_test, axis=1)

model = Sequential()
model.add(Linear(in_features=784, out_features=256))
model.add(Tanh())
model.add(Dropout(dropout_rate=0.4))
model.add(Linear(in_features=256, out_features=128))
model.add(Tanh())
model.add(Linear(in_features=128, out_features=10))
model.add(Softmax())

epochs = 50
batch_size = 1
learning_rate = 0.001

for epoch in range(epochs):
    total_loss = 0
    correct_cnt = 0
    for batch_start in range(0, len(x_train), batch_size):
        batch_end = batch_start + batch_size
        x_batch = x_train[batch_start:batch_end]
        y_batch = y_train[batch_start:batch_end]

        output = model.forward(x_batch, is_training=True)
        loss, grad_loss = cross_entropy_loss(output, y_batch)
        total_loss += loss

        model.backward(grad_loss, learning_rate)
        correct_cnt += np.sum(np.argmax(output, axis=1) == np.argmax(y_batch, axis=1))

    test_output = model.forward(x_test, is_training=False)
    test_accuracy = np.mean(np.argmax(test_output, axis=1) == y_test_labels)
    train_accuracy = correct_cnt / len(x_train)

    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}, Train Acc: {train_accuracy:.4f}, Test Acc: {test_accuracy:.4f}")

Epoch 1, Loss: 60861.5794, Train Acc: 0.8321, Test Acc: 0.9121
Epoch 2, Loss: 36083.3105, Train Acc: 0.9032, Test Acc: 0.9272
Epoch 3, Loss: 32346.7095, Train Acc: 0.9144, Test Acc: 0.9334
Epoch 4, Loss: 29969.7811, Train Acc: 0.9214, Test Acc: 0.9370
Epoch 5, Loss: 28198.1740, Train Acc: 0.9274, Test Acc: 0.9415
Epoch 6, Loss: 26581.9031, Train Acc: 0.9320, Test Acc: 0.9451
Epoch 7, Loss: 25157.8173, Train Acc: 0.9356, Test Acc: 0.9482
Epoch 8, Loss: 23879.8451, Train Acc: 0.9397, Test Acc: 0.9503
Epoch 9, Loss: 22700.7032, Train Acc: 0.9428, Test Acc: 0.9532
Epoch 10, Loss: 21727.8395, Train Acc: 0.9457, Test Acc: 0.9550
Epoch 11, Loss: 20815.1246, Train Acc: 0.9477, Test Acc: 0.9568
Epoch 12, Loss: 20021.1582, Train Acc: 0.9497, Test Acc: 0.9589
Epoch 13, Loss: 19135.2076, Train Acc: 0.9523, Test Acc: 0.9619
Epoch 14, Loss: 18508.8868, Train Acc: 0.9534, Test Acc: 0.9627
Epoch 15, Loss: 17839.5272, Train Acc: 0.9556, Test Acc: 0.9642
Epoch 16, Loss: 17202.4393, Train Acc: 0.9573, Te

In [15]:
import json

weights_data = {
    "weight_1": {
        "weight": model.layers[0].weights.tolist(),
        "bias": model.layers[0].bias.tolist()
    },
    "weight_2":{
        "weight": model.layers[3].weights.tolist(),
        "bias": model.layers[3].bias.tolist()
    },
    "weight_3": {
        "weight": model.layers[5].weights.tolist(),
        "bias": model.layers[5].bias.tolist()
    }
}

with open("../backend/app/weights/weights.json", 'w') as f:
    json.dump(weights_data, f, indent=4)

print("Веса успешно сохранены в файл")

Веса успешно сохранены в файл


In [38]:
from PIL import Image
import numpy as np


def load_and_preprocess_image(image_path, target_size=(28, 28)):
    image = Image.open(image_path).convert('L')
    image = image.resize(target_size)
    image_array = np.array(image)

    if np.mean(image_array) > 127:
        image_array = 255 - image_array

    image_array = image_array / 255.0

    image_array = image_array.flatten().reshape(1, -1)

    return image_array


def predict_image(model, image_path):
    processed_image = load_and_preprocess_image(image_path)
    output = model.forward(processed_image, is_training=False)

    probabilities = np.exp(output)

    predicted_class = np.argmax(probabilities, axis=1)[0]

    return predicted_class, probabilities


image_path = r"C:\Users\vgrig\Desktop\Untitled.png"

predicted_class, probabilities = predict_image(model, image_path)

print(f"Predicted class: {predicted_class}")
print(f"Probabilities: {probabilities}")

Predicted class: 1
Probabilities: [[1.05299529 1.30401587 1.0440659  1.05355725 1.0539827  1.09212795
  1.07291014 1.10848117 1.12234448 1.17131655]]
